# QV on Deneb

In [1]:
%load_ext autoreload
%autoreload 2

## Choose Deneb as a backend

In [2]:
backend = "deneb"

## Set IQM Token if using Resonance.

In [3]:
import os
os.environ["IQM_TOKEN"] = ""

## Quantum Volume Configuration

In [48]:
from iqm.benchmarks.quantum_volume.quantum_volume import *

In [6]:
EXAMPLE_QV = QuantumVolumeConfiguration(
    num_circuits=500,
    shots=2**8,
    calset_id=None,
    num_sigmas=2,
    choose_qubits_routine="custom",
    custom_qubits_array=[[1,2,4]],  ### specify the layout of the qubits. 0 is reserved for the resonator.
    qiskit_optim_level=3,
    optimize_sqg=True,
    routing_method="sabre",
    physical_layout="fixed",
    max_gates_per_batch=60_000, ## this needs to be reduced when running for larger number of qubits
    rem=True,
    mit_shots=1_000,
)

## Run the experiment

In [7]:
benchmark_qv = QuantumVolumeBenchmark(backend, EXAMPLE_QV)
run0_qv = benchmark_qv.run()

2024-12-23 13:13:25,269 - iqm.benchmarks.logging_config - INFO - Executing QV on qubits [1, 2, 4]
2024-12-23 13:13:25,799 - iqm.benchmarks.logging_config - INFO - Successfully generated all 500 circuits to be executed
2024-12-23 13:13:25,799 - iqm.benchmarks.logging_config - INFO - Will transpile according to "fixed" physical layout
2024-12-23 13:13:25,799 - iqm.benchmarks.logging_config - INFO - Transpiling for backend IQM Backend with optimization level 3, sabre routing method and SQG optimization all circuits
2024-12-23 13:13:46,649 - iqm.benchmarks.logging_config - INFO - Submitting batch with 500 circuits corresponding to qubits [1, 2, 4]
2024-12-23 13:13:46,668 - iqm.benchmarks.logging_config - INFO - max_gates_per_batch restriction: submitting subbatch #1 with 500 circuits corresponding to qubits [1, 2, 4]
C:\Users\PedroFigueroaRomero\anaconda3\envs\iqm-benchmarks\Lib\site-packages\iqm\qiskit_iqm\iqm_provider.py:170: UserWarning: Unknown backend option(s): {'calibration_set_id'}

## Perform Analysis

In [9]:
result0_qv = benchmark_qv.analyze()

2024-12-23 13:15:52,651 - iqm.benchmarks.logging_config - INFO - Noiseless simulation and post-processing for layout [1, 2, 4]
2024-12-23 13:15:54,360 - iqm.benchmarks.logging_config - INFO - REM post-processing for layout [1, 2, 4] with 1000 shots


List all the keys in the attributes of the dataset

In [11]:
attr_keys = sorted([str(x) for x in list(result0_qv.dataset.attrs.keys())])
for k in attr_keys:
    print(k)

0
REM_quasidistributions
backend_configuration_name
backend_name
benchmark
calset_id
choose_qubits_routine
custom_qubits_array
execution_timestamp
max_gates_per_batch
mit_shots
num_circuits
num_sigmas
optimize_sqg
physical_layout
qiskit_optim_level
rem
routing_method
session_timestamp
shots


The data for specific experiments is indexed by 0,1,..., e.g., for the first experiment:

In [12]:
for k in result0_qv.dataset.attrs[0].keys():
    print(k)

qubits
qv_results_type
time_circuit_generation
time_transpilation
time_batching
time_submit
time_retrieve
all_job_metadata
operation_counts
cumulative_average_heavy_output_probability
cumulative_stddev_heavy_output_probability
heavy_output_probabilities
REM_cumulative_average_heavy_output_probability
REM_cumulative_stddev_heavy_output_probability
REM_heavy_output_probabilities


The observation object contains the benchmark results

In [ ]:
result0_qv.observations

Or print all the plots at once

In [ ]:
result0_qv.plot_all()

Store all the data of the run if one sees a successful quantum volume. 


In [ ]:
## Things needed to load the data. 
HOP = result0_qv.dataset.attrs[0]["REM_heavy_output_probabilities"]
HOP_cumul = result0_qv.dataset.attrs[0]["REM_cumulative_average_heavy_output_probability"]
HOP_cumul_std = result0_qv.dataset.attrs[0]["REM_cumulative_stddev_heavy_output_probability"]
HOP_threshold = [2.0/3.0]*len(HOP)

In [57]:
# Create a directory to store the pickle files with a timestamp
import pickle
import os
from datetime import datetime

date_str = datetime.now().strftime("%Y-%m-%d")
save_dir = f"data_QV_{date_str}"
os.makedirs(save_dir, exist_ok=True)

# Function to save a list as a pickle file
def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

# Save each list separately
save_pickle(HOP, os.path.join(save_dir, "HOP.pkl"))
save_pickle(HOP_cumul, os.path.join(save_dir, "HOP_cumul.pkl"))
save_pickle(HOP_cumul_std, os.path.join(save_dir, "HOP_cumul_std.pkl"))
save_pickle(HOP_threshold, os.path.join(save_dir, "HOP_threshold.pkl"))

print(f"Pickle files saved in {save_dir}")